In [3]:
import pandas as pd
import re
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

# ---------------------- 1. 读取原始数据 ----------------------
raw_file_path = "/Users/syx/Documents/python脚本/大众点评-爬取代码/pet_funeral_shenzhen.xlsx"  # 请确认文件路径正确
if not os.path.exists(raw_file_path):
    print(f"❌ 原始文件不存在：{raw_file_path}")
    exit()
df = pd.read_excel(raw_file_path)
print("="*50)
print(f"📊 原始数据概况：总行数={len(df)}，字段={df.columns.tolist()}")
print("="*50)


# ---------------------- 2. 筛选相关商家（保留原逻辑） ----------------------
def filter_pet_funeral_merchants(df):
    core_keywords = ['宠物殡葬', '宠物火化', '宠物善后', '宠物善终','壁葬','安葬','宠物天堂','善终',
                    '宠物安葬', '宠物葬礼', '宠物纪念', '毛孩子殡葬','火化','生命纪念','宠物告别','殡仪']
    keyword_condition = (
        df['name'].astype(str).str.contains('|'.join(core_keywords), case=False) | 
        df['address'].astype(str).str.contains('|'.join(core_keywords), case=False) | 
        df['comment_num'].astype(str).str.contains('|'.join(core_keywords), case=False)
    )
    # 筛选后去重
    df_filtered = df[keyword_condition].drop_duplicates(subset=['name', 'address'], keep='first').reset_index(drop=True)
    return df_filtered

df_related = filter_pet_funeral_merchants(df)


# ---------------------- 3. 核心修复：准确统计“无评分（含0.0分）”的商家 ----------------------
def count_no_star_merchants(df):
    """统计所有无评分的商家（含0.0分、空值、“无评分”文本等）"""
    # 定义“无评分”的判断条件（覆盖所有场景）
    no_star_condition = (
        # 场景1：float格式的0.0分
        (df['star'] == 0.0) | 
        # 场景2：字符串格式的“0.0分”“0分”
        df['star'].astype(str).str.contains(r'^0\.0分$|^0分$', regex=True) | 
        # 场景3：空值、空字符串
        (df['star'].isna()) | (df['star'].astype(str) == '') | 
        # 场景4：明确的“无评分”文本
        (df['star'] == '无评分') | (df['star'] == '暂无评分')
    )
    # 统计符合条件的商家数量
    no_star_count = len(df[no_star_condition])
    # 返回数量和对应的商家数据（便于验证）
    no_star_merchants = df[no_star_condition]
    return no_star_count, no_star_merchants


# ---------------------- 4. 重新统计并验证 ----------------------
# 统计相关商家中的无评分数量
no_star_count, no_star_merchants = count_no_star_merchants(df_related)
# 统计无人均价格的数量（同步优化，避免类似错误）
no_avg_price_count = len(df_related[
    (df_related['avg_price'] == '人均 -') | (df_related['avg_price'].isna()) | (df_related['avg_price'].astype(str) == '')
])
# 统计有完整数据（有人均+有评分）的数量
complete_data_count = len(df_related) - no_avg_price_count - no_star_count


# ---------------------- 5. 保存筛选结果 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
related_file_name = f"pet_funeral_shenzhen_cleaned_{timestamp}.xlsx"
related_file_path = os.path.join(os.getcwd(), related_file_name)
df_related.to_excel(related_file_path, index=False, engine='openpyxl')


# ---------------------- 6. 输出准确结果 ----------------------
print("\n" + "="*60)
print(f"🎉 筛选完成！相关商家总行数：{len(df_related)}")
print(f"📁 保存路径：{related_file_path}")
print("\n📋 相关商家数据完整性统计（修复后）：")
print(f"1. 含'人均 -'/无人均的商家：{no_avg_price_count} 家")
print(f"2. 无评分（含0.0分）的商家：{no_star_count} 家")  # 修复后的统计结果
print(f"3. 同时有人均+有评分的商家：{complete_data_count} 家")
print("\n🔍 无评分商家预览（前3家，若有）：")
if no_star_count > 0:
    print(no_star_merchants[['name', 'avg_price', 'star', 'address']].head(3))
else:
    print("   暂无无评分的相关商家")
print("="*60)


# 可选：验证原始评分字段的格式（帮助你确认数据实际情况）
print(f"\n📊 原始评分字段（star）的唯一值示例：")
unique_stars = df_related['star'].unique()[:10]  # 打印前10个唯一值
for star in unique_stars:
    print(f"   格式={type(star).__name__}, 值='{star}'")

📊 原始数据概况：总行数=173，字段=['page', 'index', 'name', 'link', 'address', 'avg_price', 'comment_num', 'star']

🎉 筛选完成！相关商家总行数：39
📁 保存路径：/Users/syx/Documents/python脚本/pet_funeral_shenzhen_cleaned_20251115_001632.xlsx

📋 相关商家数据完整性统计（修复后）：
1. 含'人均 -'/无人均的商家：11 家
2. 无评分（含0.0分）的商家：19 家
3. 同时有人均+有评分的商家：9 家

🔍 无评分商家预览（前3家，若有）：
                   name avg_price  star address
1   意生臻宠·尊享宠物善终殡仪(深圳湾店)      人均 -  0.0分     科技园
2    念宠·宠物善终服务中心(龙华大浪店)   人均 ￥797  0.0分      大浪
12    宠星纪·宠物善终殡葬火化(龙岗店)   人均 ￥938  0.0分  罗岗/求水山

📊 原始评分字段（star）的唯一值示例：
   格式=str, 值='4.0分'
   格式=str, 值='0.0分'
   格式=str, 值='4.5分'
   格式=str, 值='5.0分'
   格式=str, 值='3.5分'


In [4]:
import pandas as pd
import os
from datetime import datetime

# ---------------------- 配置参数（请修改为你的实际路径） ----------------------
INPUT_RAW_DATA = "/Users/syx/Documents/python脚本/pet_funeral_shenzhen_cleaned_20251115_001632.xlsx"  # 深圳原始数据路径
OUTPUT_CLEANED_DATA = f"shenzhen_second_cleaned_{datetime.now().strftime('%Y%m%d')}.xlsx"  # 输出路径


# ---------------------- 核心：深圳地址精准分区函数 ----------------------
def address_to_district(address):
    """将地址映射到深圳行政区（覆盖西丽、科技园、龙华等所有提供的地点）"""
    if pd.isna(address):
        return "地址缺失"
    
    addr = str(address).strip().replace(" ", "").lower()
    if addr == "":
        return "地址缺失"

    # 深圳地点-行政区映射表（严格按深圳最新行政规划，覆盖所有提供的地点）
    district_mapping = {
        # 南山区（含西丽、科技园、南头等）
        "南山区": ["南山区", "南山", "西丽", "科技园", "南头"],
        # 福田区（含八卦岭、园岭、景田等）
        "福田区": ["福田区", "福田", "八卦岭", "园岭", "景田", "福田中心", "岗厦"],  # 梅林关主体属龙华，但常关联福田，此处按行政归属龙华，福田关键词不含梅林关
        # 龙华区（含大浪、民治、龙华等）
        "龙华区": ["龙华区", "龙华", "大浪", "民治", "梅林关"],  # 梅林关行政属龙华区
        # 龙岗区（含坂田、罗岗、大芬等）
        "龙岗区": ["龙岗区", "龙岗", "坂田", "杨美", "华南城", "龙岗万达广场", "罗岗", "求水山", "龙岗中心城区", 
                  "仁恒梦中心", "大芬", "南岭", "岗头", "雪象", "平湖"],
        # 宝安区（含沙井、固戍、西乡等）
        "宝安区": ["宝安区", "宝安", "沙井", "固戍", "石岩", "新安", "西乡"],
        # 罗湖区（含莲塘、草埔、笋岗等）
        "罗湖区": ["罗湖区", "罗湖", "莲塘", "草埔", "笋岗"],
        # 光明区（含公明）
        "光明区": ["光明区", "光明", "公明"],
        # 其他区（预留扩展，如盐田区、坪山区等）
        "盐田区": ["盐田区", "盐田"],
        "坪山区": ["坪山区", "坪山"],
        "大鹏新区": ["大鹏新区", "大鹏"]
    }

    # 精准匹配：优先匹配完整关键词（如“八卦岭/园岭”会匹配“八卦岭”或“园岭”）
    for district, keywords in district_mapping.items():
        for kw in keywords:
            if kw in addr:
                return district

    return "待确认（需人工核对）"  # 未匹配到的地址（极少）


# ---------------------- 执行清洗分区 ----------------------
if not os.path.exists(INPUT_RAW_DATA):
    print(f"❌ 错误：未找到原始数据文件，请检查路径：{INPUT_RAW_DATA}")
else:
    # 读取原始数据
    df = pd.read_excel(INPUT_RAW_DATA)
    if "address" not in df.columns:
        print(f"❌ 错误：原始数据中未找到 'address' 列，请确认列名正确")
    else:
        # 处理address列，新增行政区列（district_cn）
        df["district_cn"] = df["address"].apply(address_to_district)
        
        # 统计分区结果（方便检查准确性）
        print(f"\n📊 地址分区结果统计（共{len(df)}条数据）：")
        district_stats = df["district_cn"].value_counts()
        for dist, cnt in district_stats.items():
            print(f"   {dist}：{cnt}条（占比：{cnt/len(df)*100:.1f}%）")
        
        # 保存清洗后的数据（含行政区列）
        df.to_excel(OUTPUT_CLEANED_DATA, index=False, engine="openpyxl")
        print(f"\n✅ 清洗完成！带行政区的数据已保存至：{OUTPUT_CLEANED_DATA}")
        print(f"   （请先检查该文件中的 'district_cn' 列，确认分区正确后，再运行热力图代码）")


📊 地址分区结果统计（共39条数据）：
   龙岗区：10条（占比：25.6%）
   南山区：9条（占比：23.1%）
   龙华区：6条（占比：15.4%）
   宝安区：6条（占比：15.4%）
   福田区：4条（占比：10.3%）
   罗湖区：3条（占比：7.7%）
   光明区：1条（占比：2.6%）

✅ 清洗完成！带行政区的数据已保存至：shenzhen_second_cleaned_20251115.xlsx
   （请先检查该文件中的 'district_cn' 列，确认分区正确后，再运行热力图代码）
